In [1]:
import logging
import datetime

import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

import modules.brevitas.model_mobilenetv2_mini_Resnet_Brevitas as cnv_model

import modules.loss as loss
import modules.metrics as metrics
import modules.val_epoch as val_epoch
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


# Logger

In [2]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info(f'{config.MODEL} Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tBrevitas Default.\n'+ 
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [3]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')


Datasets Length
	Train and Val: 128

Load Model: True
	Model: ./experiments_brevitas/test_v05_mini_resnet_70k_full_ds/weights/MY_MBLNET_V2_RESNET_classifier__best_mean_F1.pt
Device: cuda
Batch Size: 64
Num Workers: 8
Pin Memory: True

IMG DIMS:
	Width: 224
	Height: 224


# Dataloaders

In [4]:
val_loader = dataloaders.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 62
DFire only smoke images: 31
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 128

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 57
FASDD only smoke images: 28
FASDD only fire images: 1
FASDD smoke and fire images: 42

Test FASDD UAV dataset len: 128

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 59
FASDD only smoke images: 31
FASDD only fire images: 12
FASDD smoke and fire images: 26

Test FASDD CV dataset len: 128

==================== CONCATENATE
Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 256
Concatenate with FASDD CV dataset
Test dataset len: 384


### Datasets Length

In [5]:
logger.info("\n********* Datasets Length *********")

print(f'Test Dataset Length: {len(val_loader.dataset)}')
logger.info(f'Test Dataset Length: {len(val_loader.dataset)}')

Test Dataset Length: 384


# Load Model

In [6]:
model = cnv_model.MobileNetV2_MINI_RESNET().to(config.DEVICE)

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


### Parameters

In [7]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 68914
Total parameters = 68914



### Check Model Shape: Random Input

In [8]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Model shape is tensor([[ 3.6042, -3.4202],
        [ 2.7307, -3.0150],
        [ 2.3943, -2.4195],
        [ 2.8668, -3.0222]], device='cuda:0', grad_fn=<AddmmBackward0>)


### Torchinfo

In [9]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))
logger.info(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                                                           Output Shape              Param #
MobileNetV2_MINI_RESNET                                                          [1, 2]                    --
├─Sequential: 1-1                                                                [1, 64, 14, 14]           60,206
│    └─QuantIdentity: 2-1                                                        [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                       [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                       [1, 3, 224, 224]          1
├─Sequential: 1-72                                                               --                        (recursive)
│    └─QuantReLU: 2-88                                                           --                        (recursive)
│    │    └─ActQuantProxyFromInjector: 3-96                                      --           

## Load Checkpoint

In [10]:
if config.LOAD_MODEL:
    print(f'Loading Checkpoint: \n{config.LOAD_MODEL_FILE}')
    logger.info(f'Loading Checkpoint: \n{config.LOAD_MODEL_FILE}')
    model.eval();
    epochs_trained = utils.load_checkpoint(
        config.LOAD_MODEL_FILE, 
        model=model, 
        optimizer=None, 
        scheduler=None, 
        device=config.DEVICE)
    print(f'Loading model trained {epochs_trained} epochs')
    logger.info(f'Loading model trained {epochs_trained} epochs')

Loading Checkpoint: 
./experiments_brevitas/test_v05_mini_resnet_70k_full_ds/weights/MY_MBLNET_V2_RESNET_classifier__best_mean_F1.pt
Loading Model. Trained during 91 epochs
Loading model trained 91 epochs


In [11]:
model.to(config.DEVICE);
model.eval();

# Loss Function

In [12]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Eval Model before changing Weights

In [13]:
with torch.no_grad():
    _, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,
        loss_fn = loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.10it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
16.005      |10.937      |5.069       
SMOKE -> Precision: 0.9448 - Recall: 0.9096 - Accuracy: 0.9297 - F1: 0.9268
FIRE -> Precision: 0.9187 - Recall: 0.9741 - Accuracy: 0.9661 - F1: 0.9456


In [14]:
logger.info('\nTesting with FULL TEST LOADER before changing BN weights')  
logger.info(val_metrics)
f1_mean = (val_metrics["F1"][0] + val_metrics["F1"][1]) / 2
print(f'F1 Mean = {f1_mean:.4f}')
logger.info(f'F1 Mean = {f1_mean:.4f}')

F1 Mean = 0.9362


# Modify BatchNorm values under a threshold

In [15]:
### BN
# model.features[1][1].__dict__
# model.features[1][1].running_mean

In [16]:
## CONVs
# print(model.features[1][0].weight_quant.scale)
# print(model.features[1][0].weight_quant.scale())

# scale = model.features[1][0].weight_quant.scale

# print(scale < 1e-5)
# idx = torch.abs(scale) < 1e-5
# scale[idx] = 0
# print(scale)
# scale = torch.tensor(scale)
# model.features[1][0].weight_quant.scale = scale

# print(model.features[1][0].weight_quant.scale)

In [17]:
eps_weight = 1e-5
eps_bias = 1e-5
eps_mean = 1e-5
eps_var = 1e-5

eps_conv_weight = 1e-5
eps_conv_scale = 2e-5

logger.info("\nBN -> Different Epsilon for Weight, Bias, Mean and Variance")
logger.info(f'Weight = {eps_weight}')
logger.info(f'Bias = {eps_bias}')
logger.info(f'Mean = {eps_mean}')
logger.info(f'Variance = {eps_var}')

logger.info("\nConvs -> Epsilon for Weight")
logger.info(f'Weight = {eps_conv_weight}')

In [18]:
for name, mod in model.named_modules():
    if isinstance(mod, nn.BatchNorm2d):
        print(f'****** BN2d Layer {name}')
        print(f'\t_______________ Weights before changing _______________\n {mod.weight}')
        with torch.no_grad():
            idx_weight = torch.abs(mod.weight) < eps_weight
            idx_bias = torch.abs(mod.bias) < eps_bias
            idx_mean = torch.abs(mod.running_mean) < eps_mean
            idx_var = torch.abs(mod.running_var) < eps_var
            print(f'\tWeights < {eps_weight} = \n{mod.weight[idx_weight]}')
            # mod.weight[idx_weight] = eps_weight
            # mod.bias[idx_bias] = eps_bias
            # mod.running_mean[idx_mean] = eps_mean
            # mod.running_var[idx_var] = eps_var
            mod.weight[idx_weight] = 0.
            mod.bias[idx_bias] = 0.
            mod.running_mean[idx_mean] = 0.
            mod.running_var[idx_var] = 0.
            print(f'\t{idx_weight.nonzero()}')
            print(f'\tWeights after changing {mod.weight}\n')
    elif isinstance(mod, nn.Conv2d):
        print(f'++++++ Conv Layer {name}')
        print(f'\t_______________ Weights before changing _______________\n {mod.weight}')
        with torch.no_grad():
            idx_weight = torch.abs(mod.weight) < eps_conv_weight
            scale_conv = mod.weight_quant.scale()
            print(f'\tScale Conv before changing: {scale_conv}')
            idx_scale = torch.abs(scale_conv) < 1e-5
            scale_conv[idx_scale] = 2e-5
            mod.weight_quant.scale = scale_conv
            print(f'\tScale Conv after changing: {mod.weight_quant.scale}')
            print(f'\tWeights < {eps_conv_weight} = \n{mod.weight[idx_weight]}')
            # mod.weight[idx_weight] = eps_conv_weight
            mod.weight[idx_weight] = 0.
            print(f'\t{idx_weight.nonzero()}')
            print(f'\tWeights after changing {mod.weight}\n')    
    elif isinstance(mod, nn.BatchNorm1d):
        print(f'\t###### BN1d {name}')

++++++ Conv Layer features.1.0
	_______________ Weights before changing _______________
 Parameter containing:
tensor([[[[-6.0126e-02, -1.0615e-01,  9.1972e-02],
          [-2.2339e-01, -5.1461e-02,  2.8374e-01],
          [-2.2667e-01,  1.0918e-01,  1.9617e-01]],

         [[ 1.3481e-01,  6.0398e-02,  1.5257e-01],
          [-1.1046e-01, -5.6632e-02,  2.8323e-01],
          [-2.8261e-01, -1.0770e-01, -3.7233e-02]],

         [[ 1.7551e-01,  6.8620e-02,  1.4194e-01],
          [-5.1824e-02, -2.5467e-02,  2.2456e-01],
          [-2.8298e-01, -2.3712e-01, -3.8357e-02]]],


        [[[ 1.1962e-01,  3.3615e-02,  4.0833e-02],
          [ 1.0426e-01,  7.5142e-02,  6.4904e-02],
          [ 4.8764e-02,  3.3558e-02, -1.4841e-02]],

         [[-1.3300e-01, -1.1348e-01, -7.6821e-02],
          [-2.1587e-01, -1.5267e-01, -9.2460e-02],
          [-2.2790e-01, -1.6980e-01, -1.1773e-01]],

         [[ 8.4139e-02,  2.0780e-01, -1.6424e-03],
          [ 5.6314e-02,  2.2607e-01,  1.8288e-01],
          

# Test with DFire MINI Dataset: Train and Test

In [19]:
train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TRAIN DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 20
DFire only smoke images: 45
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 100

TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Whole Test Loader, to check it is the same as before changing weights

In [20]:
model.to(config.DEVICE);

In [21]:
with torch.no_grad():
    _, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,
        loss_fn = loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.20it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
16.136      |10.928      |5.208       
SMOKE -> Precision: 0.9494 - Recall: 0.8989 - Accuracy: 0.9271 - F1: 0.9235
FIRE -> Precision: 0.9262 - Recall: 0.9741 - Accuracy: 0.9688 - F1: 0.9496


In [22]:
logger.info('\nTesting with FULL TEST LOADER after changing weights')  
logger.info(val_metrics)
f1_mean = (val_metrics["F1"][0] + val_metrics["F1"][1]) / 2
print(f'F1 Mean = {f1_mean:.4f}')
logger.info(f'F1 Mean = {f1_mean:.4f}')

F1 Mean = 0.9365


### Train DFire MINI

In [23]:
with torch.no_grad():
    _, val_metrics = val_epoch.eval_fn(
        loader=train_dfire_mini_loader, 
        model=model,
        loss_fn = loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.25it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.300       |0.190       |0.110       
SMOKE -> Precision: 0.9583 - Recall: 0.9200 - Accuracy: 0.9100 - F1: 0.9388
FIRE -> Precision: 1.0000 - Recall: 0.9429 - Accuracy: 0.9800 - F1: 0.9706


In [24]:
logger.info('\nTesting with DFire MINI TRAIN after changing weights')  
logger.info(val_metrics)

### Test DFire MINI

In [25]:
with torch.no_grad():
    _, val_metrics = val_epoch.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,
        loss_fn = loss_fn,
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 50.07it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.159       |0.104       |0.056       
SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [26]:
logger.info('\nTesting with DFire MINI TEST after changing weights')  
logger.info(val_metrics)

# Convert the Model to BIPOLAR OUT

In [27]:
import brevitas.nn as qnn
import torch.nn as nn

In [28]:
class CNV_BIPOLAR_OUT(nn.Module):
    def __init__(self, base_model):
        super(CNV_BIPOLAR_OUT, self).__init__()
        self.base_model = base_model
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.base_model(x)
        x = self.qnt_output(x)
        return x

In [29]:
cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to(config.DEVICE)

### New Evaluation for BIPOLAR Out Model

In [30]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        # print(y.shape)
        # print(yhat.shape)
        
        yhat[yhat < 1] = 0
    
        metrics.precision_metric.update(yhat, y)
        metrics.recall_metric.update(yhat, y)
        metrics.accuracy_metric.update(yhat, y)
        metrics.f1_metric.update(yhat, y)
   
    precision = metrics.precision_metric.compute()
    recall = metrics.recall_metric.compute()
    accuracy = metrics.accuracy_metric.compute()
    f1 = metrics.f1_metric.compute()
    
    metrics.precision_metric.reset()
    metrics.recall_metric.reset()
    metrics.accuracy_metric.reset()
    metrics.f1_metric.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

In [31]:
logger.info("\n###############################################################")
logger.info("                 Results of BIPOLAR OUT Model")
logger.info("###############################################################")

### Full DS

In [32]:
cnv_bipolar_out.eval()
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=val_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.19it/s]

SMOKE -> Precision: 0.9494 - Recall: 0.8989 - Accuracy: 0.9271 - F1: 0.9235
FIRE -> Precision: 0.9262 - Recall: 0.9741 - Accuracy: 0.9688 - F1: 0.9496


In [33]:
logger.info('\nTesting with FULL TEST LOADER')  
logger.info(val_metrics)
f1_mean = (val_metrics["F1"][0] + val_metrics["F1"][1]) / 2
print(f'F1 Mean = {f1_mean:.4f}')
logger.info(f'F1 Mean = {f1_mean:.4f}')

F1 Mean = 0.9365


### Mini Train

In [34]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=train_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 52.81it/s]

SMOKE -> Precision: 0.9583 - Recall: 0.9200 - Accuracy: 0.9100 - F1: 0.9388
FIRE -> Precision: 1.0000 - Recall: 0.9429 - Accuracy: 0.9800 - F1: 0.9706


In [35]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
logger.info(val_metrics)

### Mini Test

In [36]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=test_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device=config.DEVICE)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 48.67it/s]

SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [37]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
logger.info(val_metrics)

# Export Bipolar to QONNX

In [38]:
save_f1_name = 'best_mean_F1'
save_bipolar_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '__BIPOLAR_Out'
utils.export_onnx(cnv_bipolar_out, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_bipolar_onnx, config.DEVICE)

Model exported to ONNX: experiments_onnx_BN/test_v06_mini_resnet_70k_CONVs_Scale_added__128_ds/onnx/Mobilenetv2_MINI_Resnet_classifier__best_mean_F1__BIPOLAR_Out
